In [1]:
import pandas as pd
train_data = pd.read_csv('../data/gaiic_track3_round1_train_20210228.tsv', sep='\t', header=None)
train_data.rename(columns={0:'sen1', 1:'sen2', 2:'labels'}, inplace=True)
train_data['labels'] = train_data['labels'].astype(int)
train_data = train_data.dropna().reset_index(drop=True)
test_data = pd.read_csv('../data/gaiic_track3_round1_testA_20210228.tsv', sep='\t', header=None)
test_data.rename(columns={0:'sen1', 1:'sen2'}, inplace=True)

all_sentences = pd.concat([train_data, test_data]).reset_index(drop=True)
all_sentences.drop_duplicates().reset_index(drop=True, inplace=True)
all_sentences['sen'] = all_sentences['sen1'] + ' ' + all_sentences['sen2']
all_sentences['sen1'] = all_sentences['sen2'] + ' ' + all_sentences['sen1']
pd.concat([all_sentences['sen'], all_sentences['sen1']]).to_csv('oppo.txt',index=False,header=None)
all_sentences

In [2]:
import tokenizers
filepath = 'oppo.txt'

bwpt = tokenizers.BertWordPieceTokenizer(vocab_file=None)
bwpt.train(
    files=[filepath],
    min_frequency=3,
    limit_alphabet=1000
)
bwpt.save_model("model")

In [3]:
import torch
torch.cuda.is_available()
from transformers import BertConfig
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('model/')
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained('../../../pre_model/chinese_wwm_ext_pytorch/')
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at ../../../pre_model/chinese_wwm_ext_pytorch/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="oppo.txt",
    block_size=128,
)

from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import TrainingArguments
from transformers import Trainer
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, mlm=True, mlm_probability=0.15
# )
data_collator = DataCollatorForWholeWordMask(
    tokenizer=tokenizer, mlm_probability=0.15
)
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_gpu_train_batch_size=64,
    save_steps=20_00,
    save_total_limit=2,
    prediction_loss_only=True,
    weight_decay=0.01,
    warmup_steps=4000,
    seed=888
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [7]:
%%time
trainer.train()
trainer.save_model("./model")

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,4.529800
1000,4.004200
1500,4.221500
2000,3.984500
2500,3.769000
3000,3.597800
3500,3.429900
4000,3.340800
4500,3.150600
5000,3.029100


CPU times: user 13h 32min 38s, sys: 3h 42min 54s, total: 17h 15min 32s
Wall time: 18h 14min 22s
